<a href="https://colab.research.google.com/github/KhanSchoolOfTechnology/LLMKHANAI/blob/main/GPT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
import time

In [ ]:
requests_per_minute = 30  # Adjust as per OpenAI's rate limits
request_interval = 60 / requests_per_minute

def initialize_translation_bot():
    print("English to Urdu Translation Bot (Type 'exit' to quit)")
    translate_to_urdu()

def translate_to_urdu():
    openai.api_key = api_key
    conversation = [{"role": "system", "content": "Translate english sentence to urdu"}]
    conversation_tokens = 0  # Keep track of the total tokens in the conversation
    last_request_time = 0

    while True:
        user_input = input("You (English): ")
        if user_input.lower() == 'exit':
            break

        # Check if adding the user input would exceed the model's limit
        if conversation_tokens + len(user_input.split()) > 4096:
            print("The conversation has reached the model's context limit. Resetting context.")
            conversation = [{"role": "system", "content": "Translate english sentence to urdu"}]
            conversation_tokens = 0

        # Handle rate limiting
        current_time = time.time()
        if current_time - last_request_time < request_interval:
            sleep_time = request_interval - (current_time - last_request_time)
            time.sleep(sleep_time)

        urdu_translation = get_urdu_translation(conversation, user_input)
        print(urdu_translation)

        # Add the Urdu translation to the conversation
        conversation.append({"role": "user", "content": user_input})
        conversation_tokens += len(user_input.split())

        conversation.append({"role": "assistant", "content": urdu_translation})
        conversation_tokens += len(urdu_translation.split())

        last_request_time = current_time

def get_urdu_translation(conversation, english_text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=conversation + [{"role": "user", "content": english_text}],
            max_tokens=60  # Adjust the token limit as needed
        )

        # Extract and return the Urdu translation from the response
        return response['choices'][0]['message']['content']
    except openai.error.OpenAIError as e:
        print(f"An error occurred: {e}")
        return "Translation error."